In [1]:
import os
import numpy as np
import pandas as pd
from astropy import units as u
from astropy.time import Time

import matplotlib.pyplot as plt

os.environ["OORB_DATA"] = "/home/moeyensj/software/oorb/share/oorb"

In [2]:
import pandas as pd
from astropy import units as u
from astropy.coordinates import SkyCoord


observations = pd.read_parquet("observations_201309.parquet")

coords = SkyCoord(ra=observations["ra"].values*u.deg, dec=observations["dec"].values*u.deg, frame="icrs")
observations["lon_galactic"] = coords.galactic.l.value
observations["lat_galactic"] = coords.galactic.b.value
observations["lon_ecliptic"] = coords.barycentrictrueecliptic.lon.value
observations["lat_ecliptic"] = coords.barycentrictrueecliptic.lat.value

# Only keep observations within some range of visits
visit_min = 6
visit_max = 20
visits = observations.groupby(by=["healpixel"])["mjd_utc"].nunique()
visits = visits[(visits >= visit_min) & (visits <= visit_max)]
observations_filtered = observations[observations['healpixel'].isin(visits.index.values)]

# Remove all observations near the galactic plane
observations_filtered = observations_filtered[(observations_filtered["lat_galactic"] >= 15) | (observations_filtered["lat_galactic"] <= -15)]

# Remove all observations far away from the ecliptic plane
observations_filtered = observations_filtered[(observations_filtered["lat_ecliptic"] <= 60) & (observations_filtered["lat_ecliptic"] >= -60)]

In [3]:
observations_filtered

,frame_id,obscode,exposure_id,filter,exposure_mjd_start,exposure_mjd_mid,exposure_duration,healpixel,obs_id,mjd_utc,ra,dec,ra_sigma,dec_sigma,mag,mag_sigma,lon_galactic,lat_galactic,lon_ecliptic,lat_ecliptic
28101,1390632,W84,c4d_130901_003155_ooi_g_d2,g,56536.022172,56536.022693,90.0,11465,c4d.229268.44.797,56536.022693,313.149715,-59.345394,0.000035,0.000034,23.671648,0.178042,337.109806,-38.399147,297.026083,-39.889215
28103,1390632,W84,c4d_130901_003155_ooi_g_d2,g,56536.022172,56536.022693,90.0,11465,c4d.229268.44.1135,56536.022693,312.991443,-59.327597,0.000026,0.000026,23.473284,0.154616,337.152111,-38.323439,296.935931,-39.844002
28104,1390632,W84,c4d_130901_003155_ooi_g_d2,g,56536.022172,56536.022693,90.0,11465,c4d.229268.44.1095,56536.022693,313.008272,-59.316254,0.000031,0.000030,23.704840,0.160683,337.164180,-38.334057,296.951612,-39.836425
28105,1390632,W84,c4d_130901_003155_ooi_g_d2,g,56536.022172,56536.022693,90.0,11465,c4d.229268.44.1075,56536.022693,313.013500,-59.311340,0.000053,0.000052,23.257390,0.174081,337.169670,-38.337625,296.957124,-39.832771
28106,1390632,W84,c4d_130901_003155_ooi_g_d2,g,56536.022172,56536.022693,90.0,11465,c4d.229268.44.1047,56536.022693,313.025591,-59.313235,0.000044,0.000044,23.271868,0.184343,337.165776,-38.343312,296.963768,-39.836725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13665858,1405306,W84,c4d_130930_061026_ooi_i_d2,i,56565.257250,56565.257771,90.0,8867,c4d.239722.18.1873,56565.257771,5.558317,-42.599820,0.000029,0.000029,19.938854,0.029307,322.075686,-73.425806,344.448805,-40.494884
13665859,1405306,W84,c4d_130930_061026_ooi_i_d2,i,56565.257250,56565.257771,90.0,8867,c4d.239722.18.1871,56565.257771,5.566702,-42.647946,0.000083,0.000083,20.562832,0.110122,321.989156,-73.384052,344.422765,-40.539182
13665860,1405306,W84,c4d_130930_061026_ooi_i_d2,i,56565.257250,56565.257771,90.0,8867,c4d.239722.18.1870,56565.257771,5.566803,-42.648505,0.000047,0.000047,21.142061,0.100398,321.988146,-73.383568,344.422464,-40.539698
13665861,1405306,W84,c4d_130930_061026_ooi_i_d2,i,56565.257250,56565.257771,90.0,8867,c4d.239722.18.1869,56565.257771,5.565718,-42.648523,0.000050,0.000050,20.068970,0.053143,321.990683,-73.383236,344.421556,-40.539298


In [4]:
from thor.orbits import Orbits
from astropy.time import Time

orbits = Orbits.fromHorizons(["2013 RR165"], Time([observations["mjd_utc"].min()], format="mjd", scale="utc"))

In [5]:
from thor import preprocessObservations

preprocessed_observations, preprocessed_associations = preprocessObservations(
    observations,
    {   
        "mjd" : "mjd_utc",
        "RA_deg" : "ra",
        "Dec_deg" : "dec",
        "RA_sigma_deg" : "ra_sigma",
        "Dec_sigma_deg" : "dec_sigma",
        "observatory_code" : "obscode",
        "obs_id" : "obs_id",
        "obj_id" : None,
    },
    mjd_scale="utc",
)

/home/moeyensj/projects/thor/thor_stable/thor/data_processing.py:152: UserWarning: No object ID column defined in the column_mapping dictionary.
Assuming no observations have been associated with a known object...

  warnings.warn(warning, UserWarning)
/home/moeyensj/projects/thor/thor_stable/thor/data_processing.py:267: UserWarning: Observation IDs should be of type string, converting...
  warnings.warn(warn)


In [6]:
0.005 * 3600

18.0

In [8]:
from thor import runTHOR
from thor.config import Configuration

cell_areas = [1, 2.5, 5, 7.5, 10, 20]
algorithms = ["dbscan", "hotspot_2d"]
#algorithms = ["hotspot_2d"]
clustering_radius = [1, 5, 10, 15]

for algorithm in algorithms:
    for radius in clustering_radius:
        for cell_area in cell_areas:
            config = Configuration(
                min_obs=6,
                min_arc_length=1.0,
                range_shift_config={"cell_area": cell_area},
                cluster_link_config={"eps" : radius/3600, "alg" : algorithm},
                iod_config={"rchi2_threshold" : 1000000},
                od_config={"rchi2_threshold" : 10},
                odp_config={"rchi2_threshold" : 10}
            )

            out_dir = f"{algorithm}_eps{radius}_ca{cell_area:.1f}"
            if not os.path.exists(out_dir):
                test_orbits, recovered_orbits, recovered_orbit_members = runTHOR(
                    preprocessed_observations, orbits, 
                    range_shift_config=config.RANGE_SHIFT_CONFIG,
                    cluster_link_config=config.CLUSTER_LINK_CONFIG,
                    iod_config=config.IOD_CONFIG,
                    od_config=config.OD_CONFIG,
                    odp_config=config.ODP_CONFIG,
                    out_dir=out_dir,
                    if_exists="erase",
                )

20:03:06.936 [WARNING] thor - Setting pipeline components to use min_obs: 6.
20:03:06.938 [WARNING] thor - Setting pipeline components to use min_arc_length: 1.0.
20:03:06.938 [WARNING] thor - Setting pipeline components to use min_obs: 6.
20:03:06.939 [WARNING] thor - Setting pipeline components to use min_arc_length: 1.0.
20:03:06.940 [WARNING] thor - Setting pipeline components to use min_obs: 6.
20:03:06.940 [WARNING] thor - Setting pipeline components to use min_arc_length: 1.0.
20:03:06.941 [WARNING] thor - Setting pipeline components to use min_obs: 6.
20:03:06.941 [WARNING] thor - Setting pipeline components to use min_arc_length: 1.0.
20:03:06.942 [WARNING] thor - Setting pipeline components to use min_obs: 6.
20:03:06.942 [WARNING] thor - Setting pipeline components to use min_arc_length: 1.0.
20:03:06.943 [WARNING] thor - Setting pipeline components to use min_obs: 6.
20:03:06.943 [WARNING] thor - Setting pipeline components to use min_arc_length: 1.0.
20:03:06.944 [WARNING]

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

thor_results = pd.read_csv("thor_results.csv", float_precision="round_trip", index_col=False)

In [2]:
thor_results

,cell_area,eps,alg,projection_time,projected_observations,clustering_time,restructuring_time,clusters,iod_time,iod_orbits,od_time,od_orbits,odp_time,odp_orbits
0,1.0,1.0,dbscan,27.007,11153,54.512,0.061,0,0.010,0,0.007,0.0,0.011,0.0
1,1.0,1.0,hotspot_2d,27.504,11153,28.377,0.038,0,0.009,0,0.007,0.0,0.010,0.0
2,1.0,5.0,dbscan,27.079,11153,54.247,0.138,80,5.113,56,7.078,7.0,53.103,7.0
3,1.0,5.0,hotspot_2d,26.066,11153,29.723,0.081,86,6.281,53,7.632,6.0,46.225,4.0
4,1.0,10.0,dbscan,28.083,11153,60.670,0.712,1240,7.272,820,47.850,13.0,57.528,10.0
5,1.0,10.0,hotspot_2d,26.876,11153,31.822,0.626,1405,24.310,935,175.440,9.0,54.003,9.0
6,1.0,15.0,dbscan,23.696,11153,62.533,0.967,4345,9.090,2917,145.080,10.0,32.522,10.0
7,1.0,15.0,hotspot_2d,24.834,11153,45.321,1.099,6450,11.871,4127,215.347,7.0,34.297,7.0
8,2.5,1.0,dbscan,27.481,27780,134.826,0.027,0,0.009,0,0.006,0.0,0.015,0.0
9,2.5,1.0,hotspot_2d,25.619,27780,56.093,0.036,0,0.009,0,0.006,0.0,0.014,0.0
